### Visualizing the distribution of the observations

### Load the required libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSSS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
#print 'keys from STAT=',STAT.keys()

#read data
filename=data_dir+'/US_Weather_%s.parquet'%file_index

df=sqlContext.read.parquet(filename)
print df.count()
df.show(5)

Populating the interactive namespace from numpy and matplotlib
12741
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[B8 59 00 58 38 5...|2002.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[90 59 10 5A 20 5...|2003.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     1|[E0 55 00 58 E0 5...|2004.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[50 57 90 55 A0 5...|2005.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     7|[A0 56 F0 56 A0 5...|2006.0|SSSBSSSS|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



### Select data for a particular station and measurement type

In [2]:
sqlContext.registerDataFrameAsTable(df,'weather')
Query="SELECT * FROM weather\n\tWHERE measurement='%s' and station='%s'"%('PRCP','US1CAMT0023')
print Query
df1 = sqlContext.sql(Query)
print df1.count(),'rows'
df1.show(2)
rows=df1.rdd.map(lambda row:unpackArray(row['vector'],np.float16)).collect()
T=np.vstack(rows)
T=T/10.  # scaling to make the temperature be in centingrates
shape(T)

SELECT * FROM weather
	WHERE measurement='PRCP' and station='US1CAMT0023'
3 rows
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    335.3|  36.398|-121.8685|       PRCP|US1CAMT0023|    35|[00 7E 00 7E 00 7...|2010.0|SSSBSSSS|
|    335.3|  36.398|-121.8685|       PRCP|US1CAMT0023|     4|[80 4C B4 5E 00 5...|2011.0|SSSBSSSS|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 2 rows



(3, 365)

In [3]:
sqlContext.registerDataFrameAsTable(df,'weather')
Query1="SELECT * FROM weather\n\tWHERE measurement='%s'"%('PRCP')
print Query1
df_test = sqlContext.sql(Query1)
print df_test.count(),'rows'
df_test.show()
rows_test=df_test.rdd.map(lambda row:(row.elevation,row.station,unpackArray(row['vector'],np.float16))).collect()
#rows_test1=rows_test.map(lambda (v,y):(v,np.hstack(y))).collect()
 #xs: [x[0] for x in xs]

                         #T=np.vstack(rows_test
#T=T/10.  # scaling to make the temperature be in centingrates
#shape(T)


import pandas as pd

df_test=pd.DataFrame(rows_test)

df_test1=df_test[2].apply(pd.Series)/10


del df_test[2]

df_test3=pd.concat([df_test,df_test1],axis=1)

df_test3

SELECT * FROM weather
	WHERE measurement='PRCP'
2926 rows
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    716.9| 36.5333|-121.2833|       PRCP|USC00043502|     9|[20 51 00 00 00 0...|1949.0|SSSBSSSS|
|    716.9| 36.5333|-121.2833|       PRCP|USC00043502|     4|[00 00 A0 53 00 0...|1950.0|SSSBSSSS|
|    335.3|  36.398|-121.8685|       PRCP|US1CAMT0023|    35|[00 7E 00 7E 00 7...|2010.0|SSSBSSSS|
|    335.3|  36.398|-121.8685|       PRCP|US1CAMT0023|     4|[80 4C B4 5E 00 5...|2011.0|SSSBSSSS|
|    335.3|  36.398|-121.8685|       PRCP|US1CAMT0023|     6|[00 00 00 00 00 0...|2012.0|SSSBSSSS|
|    111.3| 37.2319|-121.9592|       PRCP|USC00045123|     2|[00 00 00 00 F8 5...|1901.0|SSSBSSSS|
|    111.3| 37.2319|-121.9592|       PRCP|USC000451

,0,1,0,1,2,3,4,5,6,7,...,355,356,357,358,359,360,361,362,363,364
0,716.9,USC00043502,4.101562,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,716.9,USC00043502,0.000000,6.101562,0.000000,1.500000,0.000000,0.000000,0.000000,20.296875,...,0.000000,0.000000,0.000000,23.593750,0.000000,0.000000,0.000000,0.000000,3.300781,0.000000
2,335.3,US1CAMT0023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.093750,7.898438,0.000000,0.000000,33.000000,0.000000,0.000000,46.687500,6.101562,0.500000
3,335.3,US1CAMT0023,1.799805,42.906250,5.601562,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,335.3,US1CAMT0023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.601562,20.796875,54.593750,0.000000,13.703125,22.906250,0.000000,21.296875,2.000000,0.000000
5,111.3,USC00045123,0.000000,0.000000,19.093750,NaN,47.500000,22.593750,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,111.3,USC00045123,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.799805,47.500000,10.203125
7,111.3,USC00045123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.299805,0.000000,7.101562,0.000000
8,111.3,USC00045123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.799805,5.101562,0.000000,18.796875,39.406250,2.500000,1.299805,0.000000,7.601562,15.203125
9,111.3,USC00045123,0.000000,0.000000,1.299805,13.203125,0.000000,11.398438,26.703125,14.000000,...,0.000000,0.000000,0.000000,0.000000,3.800781,1.299805,5.601562,0.000000,25.406250,25.093750


In [4]:
df_test4=df_test3.iloc[:,2:]
result=df_test4.mean(axis=1)
type(result)

pandas.core.series.Series

In [5]:
df_test6=df_test3.iloc[:,:2]

In [6]:
df_test5=result.to_frame(name='Avg PRCP')
df_test5.shape

(2926, 1)

In [7]:
regression_data=pd.concat([df_test6, df_test5], axis=1)

In [8]:
regression_data.columns = ['Elevation', 'Station', 'Avg PRCP']
regression_data

,Elevation,Station,Avg PRCP
0,716.9,USC00043502,0.707520
1,716.9,USC00043502,1.016602
2,335.3,US1CAMT0023,3.181641
3,335.3,US1CAMT0023,2.443359
4,335.3,US1CAMT0023,2.531250
5,111.3,USC00045123,1.791016
6,111.3,USC00045123,2.169922
7,111.3,USC00045123,1.867188
8,111.3,USC00045123,2.814453
9,111.3,USC00045123,2.490234


In [9]:
import statsmodels.api as sm

## Regression

In [10]:
from pandas.stats.api import ols
regression = ols(y=regression_data['Avg PRCP'], x=regression_data['Elevation'])
regression

/Users/mengtingwang/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x> + <intercept>

Number of Observations:         2926
Number of Degrees of Freedom:   2

R-squared:         0.0331
Adj R-squared:     0.0328

Rmse:              0.7291

F-stat (1, 2924):   100.0409, p-value:     0.0000

Degrees of Freedom: model 1, resid 2924

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
             x     0.0006     0.0001      10.00     0.0000     0.0004     0.0007
     intercept     1.2307     0.0165      74.40     0.0000     1.1983     1.2632
---------------------------------End of Summary---------------------------------